In [1]:
%pip install pymilvus


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Connect to Milvus

In [40]:
from pymilvus import connections

connections.connect("default", host="127.0.0.1", port="19530")

### Create a Collection (like a table)

In [41]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Define schema fields
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=4)
]

schema = CollectionSchema(fields, description="Simple demo collection")

# Create collection
collection = Collection(name="demo_collection", schema=schema)

from pymilvus import utility
print(utility.list_collections())  # list all

['demo_collection']


In [42]:
print(utility.has_collection("demo_collection"))

# Get details about a specific collection
# Get collection details
collection = Collection("demo_collection")  # instantiate the collection object
print(collection.schema)                    # show the schema
print(collection.num_entities)              # number of entities
print(collection.description)               # optional

True
{'auto_id': True, 'description': 'Simple demo collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 200}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4}}], 'enable_dynamic_field': False}
0
Simple demo collection


### Insert Data (like INSERT INTO)

In [43]:
data = [
    ["Intro to AI", "Deep Learning", "Vector Databases"],
    [
        [0.1, 0.2, 0.3, 0.4],
        [0.2, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.6],
    ]
]

collection.insert(data)
collection.flush()
print(f"Number of entities after insertion: {collection.num_entities}")


Number of entities after insertion: 3


### Create an Index (for fast vector search)

In [44]:
index_params = {
    "metric_type": "L2",      # or "COSINE"
    "index_type": "IVF_FLAT", # fast approximate search
    "params": {"nlist": 64}
}
collection.create_index(field_name="embedding", index_params=index_params)


Status(code=0, message=)

### Query Metadata (like SQL SELECT * WHERE)

In [ ]:
collection.load()
results = collection.query(
    expr="",
    # expr="title == 'Deep Learning'",
    output_fields=["title", "embedding"],
    limit=10
)

for result in results:
    print(result)


2025-11-08 07:15:03,743 [ERROR][handler]: RPC error: [load_collection], <MilvusException: (code=100, message=collection not found[collection=462058208105097137])>, <Time:{'RPC start': '2025-11-08 07:15:03.738988', 'RPC error': '2025-11-08 07:15:03.743582'}> (decorators.py:253)


MilvusException: <MilvusException: (code=100, message=collection not found[collection=462058208105097137])>

### Search by Vector (the heart of Milvus)

In [46]:
import numpy as np

query_vector = [[0.15, 0.25, 0.35, 0.45]]

search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

results = collection.search(
    data=query_vector,
    anns_field="embedding",
    param=search_params,
    limit=2,
    output_fields=["title"]
)

for hit in results[0]:
    print(f"Matched: {hit.entity.get('title')}, Distance: {hit.distance}")


Matched: Intro to AI, Distance: 0.009999996051192284
Matched: Deep Learning, Distance: 0.01000000350177288


### List & Drop Collections

In [47]:
from pymilvus import utility

print(f"List of collection BEFORE dropping the collection: {utility.list_collections()}")   # list all
utility.drop_collection("demo_collection")  # delete one
print(f"List of collection after dropping the collection: {utility.list_collections()}")  # list all again to confirm deletion

List of collection BEFORE dropping the collection: ['demo_collection']
List of collection after dropping the collection: []
